# Rodrigo Merino de la Parra A00836396

# Simulacion de interseccion con semaforos inteligentes

Jupyter Notebook

LIBRERIAS 

pip install --upgrade mesa (Install Mesa)

pip install seaborn ( used for data visualization )

link video de simulacion: 

In [ ]:
import mesa
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid
import seaborn as sns
import numpy as np 
import pandas as pd
import random

class IntersectionModel(mesa.Model):
    '''
    Modela una interseccion controlada por semaforos inteligentes
    '''
    

    def __init__(self, width, height):
        super().__init__()
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)
        self.directions = ['N_S', 'S_N', 'W_E', 'E_W']
        self.width = width
        self.height = height
        self.traffic_lights = {}

        # Creamos los semaforos
        traffic_light_E_W = TrafficLight(1, self)
        traffic_light_W_E = TrafficLight(2, self)
        traffic_light_N_S = TrafficLight(3, self)
        traffic_light_S_N = TrafficLight(4, self)


        # Ponemos los semaforos en las posiciones correctas
        self.grid.place_agent(traffic_light_W_E, (self.width // 2 - 2, self.height // 2 - 1))  # left
        self.grid.place_agent(traffic_light_S_N, (self.width // 2, self.height // 2 - 2))  # bottom 
        self.grid.place_agent(traffic_light_E_W, (self.width // 2 + 1, self.height // 2))  # right
        self.grid.place_agent(traffic_light_N_S, (self.width // 2 - 1, self.height // 2 + 1))  # top 

        # Almacena los semaforos en un diccionario
        self.traffic_lights['W_E'] = traffic_light_W_E
        self.traffic_lights['N_S'] = traffic_light_N_S
        self.traffic_lights['E_W'] = traffic_light_E_W
        self.traffic_lights['S_N'] = traffic_light_S_N

        # Schedule a los semaforos
        self.schedule.add(traffic_light_W_E)
        self.schedule.add(traffic_light_S_N)
        self.schedule.add(traffic_light_E_W)
        self.schedule.add(traffic_light_N_S)

        '''
        self.schedule.add(arriba_izquierda)
        self.schedule.add(arriba_derecha)
        self.schedule.add(abajo_izquierda)
        self.schedule.add(abajo_derecha)
        '''


        '''
        # Set traffic light positions for the dictionary
        self.traffic_lights['N_S'] = (self.width // 2 - 1, self.height // 2 + 1)
        self.traffic_lights['W_E'] = (self.width // 2 - 2, self.height // 2 - 1)
        self.traffic_lights['E_W'] = (self.width // 2 + 1, self.height // 2)
        self.traffic_lights['S_N'] = (self.width // 2, self.height // 2 - 2)
        '''

        self.spawn_vehicle()
    
    def spawn_vehicle(self):
        # Seleccionamos una direccion aleatoria
        direction = random.choice(self.directions)
        vehicle = Vehicle(6, self, direction) # Creamos el vehiculo
        coordinate = self.get_starting_coordinate(direction)
        x, y = coordinate
        self.grid.place_agent(vehicle, (x, y))


        # Agrergamos el vehiculo al schedule
        self.schedule.add(vehicle) 

        # Creamos un agente central para controlar los semaforos
        central_controller = CentralController(5, self)  
        self.schedule.add(central_controller)
        



    # Funcion para obtener la coordenada de inicio del vehiculo
    def get_starting_coordinate(self, direction):
        if direction == 'N_S':
            return (self.width // 2 - 1, self.height - 1)
        elif direction == 'S_N':
            return (self.width // 2, 0)
        elif direction == 'W_E':
            return (0, (self.height // 2) - 1)
        elif direction == 'E_W':
            return (self.width - 1, self.height // 2)


    def step(self):
        self.schedule.step()

        # Verifica si el step actual es divisible por 3 para spawnear un nuevo vehículo
        if self.schedule.steps % 5 == 0:
            self.spawn_vehicle()




class Vehicle(mesa.Agent):
    '''
    Agente de vehiculo
    '''

    def __init__(self, unique_id, model, direction):
        #super().__init__(unique_id, model, direction)
        self.unique_id = unique_id
        self.model = model
        self.direction = direction
        self.pos = (1,1)


    def move(self):
        # Posicion a donde se movera el agente
        next_position = self.pos

        # Movemos el vehículo dependiendo de su dirección
        if self.can_move():

            if self.direction == 'N_S':
                next_position = (self.pos[0], self.pos[1] - 1)
            elif self.direction == 'S_N':
                next_position = (self.pos[0], self.pos[1] + 1)
            elif self.direction == 'W_E':
                next_position = (self.pos[0] + 1, self.pos[1])
            elif self.direction == 'E_W':
                next_position = (self.pos[0] - 1, self.pos[1])
                    

            
            # Checamos si la siguiente posición está dentro de los límites del grid
            if 0 <= next_position[0] < self.model.grid.width and 0 <= next_position[1] < self.model.grid.height:
                # Movemos el agente a la siguiente posición
                self.model.grid.move_agent(self, next_position)

                    # Verifica si la celda destino ya está ocupada por otro vehículo
                cell_contents = self.model.grid.get_cell_list_contents(next_position)
                vehicles_in_cell = [obj for obj in cell_contents if isinstance(obj, Vehicle)]

                if not vehicles_in_cell:  # Si no hay vehículos en la celda destino
                    self.model.grid.move_agent(self, next_position)

            else:
                # Si la celda destino está fuera de los límites del grid, elimina el agente
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)
        else:
            pass # se espera



    def can_move(self):
        # Coordenadas del cruzamiento
        arriba_izquierda = (self.model.width // 2 - 1, self.model.height // 2)
        arriba_derecha = (self.model.width // 2, self.model.height // 2)
        abajo_izquierda = (self.model.width // 2 - 1, self.model.height // 2 - 1)
        abajo_derecha = (self.model.width // 2, self.model.height // 2 - 1)

        # Verifica si el vehículo está en alguna de las coordenadas del cruce
        esta_en_cruce = self.pos in [arriba_izquierda, arriba_derecha, abajo_izquierda, abajo_derecha]

        # Si el vehículo está en el cruce, puede moverse sin importar el estado del semáforo
        if esta_en_cruce:
            return True

        # Obtiene el semáforo relevante basado en la dirección del vehículo.
        traffic_light = self.model.traffic_lights[self.direction]

        # Verifica si el semáforo está en rojo.
        is_light_red = traffic_light.state == 'red'

        # Calcula si el vehículo está adyacente al semáforo.
        is_adjacent_to_light = self.get_distance() == 1

        # Si el semáforo está en rojo y el vehículo está adyacente a él, no puede moverse.
        if is_light_red and is_adjacent_to_light:
            return False

        # En cualquier otro caso, el vehículo puede moverse.
        return True




    def get_distance(self):
        '''
        Funcion para calcular la distancia hacia el semaforo
        
        Regresa un numero entero

        Se puede utilizar para despues comunicarselo a los semaforos y ir restandole en la funcion de move que se
        hara a continuacion.
        '''
        # Obtenemos la posicion del semaforo 
        traffic_light_pos = self.model.traffic_lights[self.direction].pos
        # Calculamos distancia Manhattan
        distance = abs(self.pos[0] - traffic_light_pos[0]) + abs(self.pos[1] - traffic_light_pos[1])
        return distance

    def step(self):
        self.move()





class TrafficLight(mesa.Agent):
    '''
    Agente de semaforo inteligente
    '''

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.state = 'yellow' # Estado inicial
        self.phase = None # Fase actual
        self.phase_counter = 0 # Contador de pasos en la fase actual

    def change_state(self):
        pass  
    def step(self):
        pass
        
class CentralController(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.traffic_light_schedule = None  # Schedule para los semaforos
    
    def step(self):
        self.manage_traffic_lights()
        
        


class CentralController(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.phase = None  # Fase actual
        self.phase_counter = 0  # Contador de pasos en la fase actual

    def step(self):
        self.manage_traffic_lights()

    def manage_traffic_lights(self):
        traffic_light_N_S = self.model.traffic_lights['N_S']
        traffic_light_S_N = self.model.traffic_lights['S_N']
        traffic_light_E_W = self.model.traffic_lights['E_W']
        traffic_light_W_E = self.model.traffic_lights['W_E']

        traffic_light_N_S_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_N_S.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'N_S']
        traffic_light_S_N_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_S_N.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'S_N']
        traffic_light_E_W_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_E_W.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'E_W']
        traffic_light_W_E_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_W_E.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'W_E']

        
        # Comprueba si hay vehiculos esperando en alguna dirección y cambia los semáforos correspondientes a verde
        if traffic_light_N_S_neighbors or traffic_light_S_N_neighbors:
            # Si hay vehiculos esperando en dirección N_S o S_N, cambia esos semáforos a verde
            traffic_light_N_S.state = 'green'
            traffic_light_S_N.state = 'green'
            traffic_light_E_W.state = 'red'  # Cambia a rojo para evitar conflictos
            traffic_light_W_E.state = 'red'
            self.phase = 1
        elif traffic_light_E_W_neighbors or traffic_light_W_E_neighbors:
            # Si hay vehiculos esperando en direccion E_W o W_E, cambia esos semaforos a verde
            traffic_light_E_W.state = 'green'
            traffic_light_W_E.state = 'green'
            traffic_light_N_S.state = 'red'
            traffic_light_S_N.state = 'red'
            self.phase = 1

        if self.phase is not None:
            # Si no hay vehiculos esperando, sigue con la lógica de programación fija
            if self.model.schedule.steps % 10 < 5:
                traffic_light_N_S.state = 'green'
                traffic_light_S_N.state = 'green'
                traffic_light_E_W.state = 'red'
                traffic_light_W_E.state = 'red'
            else:
                traffic_light_N_S.state = 'red'
                traffic_light_S_N.state = 'red'
                traffic_light_E_W.state = 'green'
                traffic_light_W_E.state = 'green'
    
    '''
    def manage_traffic_lights(self):
        traffic_light_N_S = self.model.traffic_lights['N_S']
        traffic_light_S_N = self.model.traffic_lights['S_N']
        traffic_light_E_W = self.model.traffic_lights['E_W']
        traffic_light_W_E = self.model.traffic_lights['W_E']

        traffic_light_N_S_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_N_S.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'N_S']
        traffic_light_S_N_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_S_N.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'S_N']
        traffic_light_E_W_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_E_W.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'E_W']
        traffic_light_W_E_neighbors = [carro for carro in self.model.grid.get_neighbors(traffic_light_W_E.pos, moore=False, include_center=True, radius=3) if isinstance(carro, Vehicle) and carro.direction == 'W_E']

        # Comprueba si hay vehículos esperando en alguna dirección y cambia los semáforos correspondientes a verde
        if traffic_light_N_S_neighbors or traffic_light_S_N_neighbors:
            # Si hay vehículos esperando en dirección N_S o S_N, cambia esos semáforos a verde
            traffic_light_N_S.state = 'green'
            traffic_light_S_N.state = 'green'
            traffic_light_E_W.state = 'red'  # Cambia a rojo para evitar conflictos
            traffic_light_W_E.state = 'red'
        elif traffic_light_E_W_neighbors or traffic_light_W_E_neighbors:
            # Si hay vehículos esperando en dirección E_W o W_E, cambia esos semáforos a verde
            traffic_light_E_W.state = 'green'
            traffic_light_W_E.state = 'green'
            traffic_light_N_S.state = 'red'
            traffic_light_S_N.state = 'red'
        else:
            # Si no hay vehículos esperando, sigue con la lógica de programación fija
            if self.model.schedule.steps % 10 < 5:
                traffic_light_N_S.state = 'yellow'
                traffic_light_S_N.state = 'yellow'
                traffic_light_E_W.state = 'yellow'
                traffic_light_W_E.state = 'yellow'
            else:
                traffic_light_N_S.state = 'yellow'
                traffic_light_S_N.state = 'yellow'
                traffic_light_E_W.state = 'yellow'
                traffic_light_W_E.state = 'yellow'
    '''
        
        


In [ ]:

def agent_portrayal(agent):
    """
    Funcion para definir como se vera cada agente en el tablero
    """
    portrayal = {"Shape": "rect", "w": 0.8, "h": 0.8, "Filled": "true"}

    if isinstance(agent, Vehicle):
        portrayal["Color"] = "blue"
        portrayal["Layer"] = 1
    elif isinstance(agent, TrafficLight):
        if agent.state == 'yellow':
            portrayal["Color"] = "yellow"
        elif agent.state == 'red':
            portrayal["Color"] = "red"
        elif agent.state == 'green':
            portrayal["Color"] = "green"
        portrayal["Layer"] = 2
        portrayal["w"] = 0.5
        portrayal["h"] = 0.5
    return portrayal

# Definir tamaño del multi grid 
grid_width = 24
grid_height = 24

grid = CanvasGrid(agent_portrayal, grid_width, grid_height, 500, 500)

# Parametros del modelo
model_params = {
    "width": grid_width,
    "height": grid_height
}

# Crear y lanzar el servidor
server = ModularServer(IntersectionModel, [grid], "Intersection Model", model_params)

server.port = 8521 # El puerto predeterminado
server.launch()